In [1]:
# <img src="./assets-resources/pydantic.png" width=400> </img>
# # OpenAI Functions 

# Allow you to format the output of an llm to input to a function. Meaning allows you to call functions from outputs of LLMs.
# Below is a traditional [openai function for function calling](https://platform.openai.com/docs/guides/function-calling).

functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [2]:
# We can bind this function to a model using the `.bind` method from the ChatOpenAI object (or any `ChatModel` object in langchain for that matter.)

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


prompt = ChatPromptTemplate.from_messages(
    [
     ("human", "{input}")   
    ]
)


In [8]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106",temperature=0).bind(functions=functions)

In [9]:
# now we can create our runnable with:
runnable = prompt | model

In [10]:
# and we can call it:
runnable.invoke({"input": "What's the weather like in Lisbon?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'weather_search'}})

In [11]:
# Let's update the `functions` variable and add another function.

# Let's say search for relevant resources given a query.
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
    {
      "name": "resources_search",
      "description": "Search for relevant resources given a query",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "The query describing the subject of the resources"
          },
        },
        "required": ["query"]
      }
    }
  ]

In [12]:
# Now we can take that same model and bind these functions to it:
model = model.bind(functions=functions)
runnable = prompt | model
runnable.invoke({"input": "What are the most useful resources of the langchain Python library?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"langchain Python library"}', 'name': 'resources_search'}})

In [13]:
# What if we wanted the model to put a json object rather than some string or some object?
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
import json

llm_chat = ChatOpenAI(temperature=0)
output_parser = StrOutputParser()

simple_chain = llm_chat | output_parser | json.loads

simple_chain.invoke("What are fun five names for dogs?The output should be a json object.")


{'names': ['Biscuit', 'Coco', 'Max', 'Lola', 'Charlie']}

In [14]:
# Below we add a list of other runnables. 

# If there is an error it will go through the list of fallback chains (meaning, other ways of processing the input in a useful way.)
# final_chain = simple_chain.with_fallbacks([simple_chain])
# Now, let's add a prompt template to this, and explore the `.batch` option of a runnable.

In [15]:
from langchain.prompts import ChatPromptTemplate

template = "Create a list of 3 exercises to drill this concept: {concept}"

prompt = ChatPromptTemplate.from_template(template)

llm_chat = ChatOpenAI(temperature=0)

output_parser = StrOutputParser()

chain = prompt | llm_chat | output_parser

chain.batch([{"concept": "derivatives"}, {"concept": "linear regression"}])

['1. Power Rule Drill: Choose a variety of functions and have students practice finding the derivative using the power rule. For example, give them functions like f(x) = x^3, g(x) = 2x^5, and h(x) = 4x^2. Have them find the derivative of each function and explain the steps they took to arrive at the answer.\n\n2. Chain Rule Drill: Provide students with composite functions and ask them to find the derivative using the chain rule. For instance, give them functions like f(x) = (2x^2 + 3x)^4 or g(x) = sin(3x^2). Have them break down the functions into their individual components and apply the chain rule to find the derivative.\n\n3. Product Rule Drill: Give students functions that require the product rule to find the derivative. For example, provide them with functions like f(x) = x^2 * sin(x) or g(x) = (2x + 1)(3x - 2). Have them apply the product rule to find the derivative and simplify the expression as much as possible.',
 '1. Simple Linear Regression Exercise:\n- Choose a dataset with

In [16]:
# We can stream results:

import matplotlib.pyplot as plt

outputs = []
streaming_output = ""
for t in chain.stream({"concept": "derivatives"}):
    print(t)


1
.
 Power
 Rule
 Drill
:
 Choose
 a
 variety
 of
 functions
 and
 have
 students
 practice
 finding
 the
 derivative
 using
 the
 power
 rule
.
 For
 example
,
 give
 them
 functions
 like
 f
(x
)
 =
 x
^
3
,
 g
(x
)
 =
 
2
x
^
5
,
 and
 h
(x
)
 =
 
4
x
^
2
.
 Have
 them
 find
 the
 derivative
 of
 each
 function
 step
 by
 step
,
 emphasizing
 the
 application
 of
 the
 power
 rule
.


2
.
 Chain
 Rule
 Drill
:
 Provide
 students
 with
 composite
 functions
 and
 ask
 them
 to
 find
 the
 derivative
 using
 the
 chain
 rule
.
 For
 instance
,
 give
 them
 functions
 like
 f
(x
)
 =
 (
2
x
^
3
 +
 
5
x
)^
4
 or
 g
(x
)
 =
 sin
(
3
x
^
2
).
 Have
 them
 break
 down
 the
 functions
 into
 their
 inner
 and
 outer
 functions
,
 apply
 the
 chain
 rule
,
 and
 simplify
 the
 derivatives
.


3
.
 Product
 and
 Qu
ot
ient
 Rule
 Drill
:
 Give
 students
 functions
 that
 require
 the
 product
 or
 quotient
 rule
 to
 find
 the
 derivative
.
 For
 example
,
 provide
 them
 with
 functions
 l